In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

# Define topography data
z = np.random.normal(size=(50,50))

# Add sinusoidal noise to topography data
def add_noise(z, amplitude):
    nx, ny = z.shape
    x = np.linspace(0, 10, num=nx)[:, None]
    y = np.linspace(0, 10, num=ny)[None, :]
    return z + amplitude * np.sin(x) + amplitude * np.sin(y)

# Define function to find local minima
def islocalminima(z, idx, jdx):
    nx, ny = z.shape
    # Check for edge cases
    if idx == 0 or idx == nx-1 or jdx == 0 or jdx == ny-1:
        return False
    center = z[idx, jdx]
    left = center <= z[idx-1, jdx]
    right = center <= z[idx+1, jdx]
    top = center <= z[idx, jdx+1]
    bottom = center <= z[idx, jdx-1]
    return left and right and top and bottom

# Define function to compute probability of local minima
def prob_local_minima(num_samples, distal):
    idx_min = []
    jdx_min = []
    for i in range(num_samples):
        idx = np.random.randint(1, z.shape[0]-1)
        jdx = np.random.randint(1, z.shape[1]-1)
        if islocalminima(z, idx, jdx) and idx > distal and len(idx_min) < 2:
            idx_min.append(idx)
            jdx_min.append(jdx)
    prob = len(idx_min) / num_samples
    return prob, idx_min, jdx_min

# Define function to update plot
def update_plot(num_samples, distal, amplitude):
    z_noise = add_noise(z, amplitude)
    prob, idx_min, jdx_min = prob_local_minima(num_samples, distal)
    fig, ax = plt.subplots()
    ax.imshow(z_noise, cmap='gray')
    ax.plot(jdx_min, idx_min, 'ro', label='Local Minima')
    ax.legend()
    plt.show()
    print(f"Probability of local minima: {prob}")

# Create slider widgets
sample_slider = IntSlider(min=10, max=1000, step=10, value=100)
distal_slider = IntSlider(min=10, max=40, step=5, value=20)
amp_slider = FloatSlider(min=0.0, max=1.0, step=0.05, value=0.1)

# Create interactive plot
interact(update_plot, num_samples=sample_slider, distal=distal_slider, amplitude=amp_slider);



interactive(children=(IntSlider(value=100, description='num_samples', max=1000, min=10, step=10), IntSlider(va…

In [ ]:
import numpy as np

def islocalminima_3d(data, x, y, z, window_size, step_size, min_distal, max_distal, n):
    # Define function to check if a point is a local minimum
    def islocalminima(data, x, y, z, i, j, k, window_size):
        center = data[i, j, k]
        neighborhood = data[i-window_size:i+window_size+1, j-window_size:j+window_size+1, k-window_size:k+window_size+1]
        if center == np.min(neighborhood):
            return True
        else:
            return False

    # Define function to check if a point is a distal local minimum
    def isdistallocminima(data, x, y, z, i, j, k, min_distal, max_distal):
        center = data[i, j, k]
        distal_cube = data[i-min_distal:i+min_distal+1, j-min_distal:j+min_distal+1, k-min_distal:k+min_distal+1]
        approximal_cube = data[i-max_distal:i+max_distal+1, j-max_distal:j+max_distal+1, k-max_distal:k+max_distal+1]
        if center == np.min(distal_cube) and center < np.min(approximal_cube):
            return True
        else:
            return False

    # Generate random samples
    i_samples = np.random.choice(range(window_size, data.shape[0]-window_size, step_size), size=n)
    j_samples = np.random.choice(range(window_size, data.shape[1]-window_size, step_size), size=n)
    k_samples = np.random.choice(range(window_size, data.shape[2]-window_size, step_size), size=n)

    # Check each sample
    localminima = []
    distallocminima = []
    for i, j, k in zip(i_samples, j_samples, k_samples):
        if islocalminima(data, x, y, z, i, j, k, window_size):
            localminima.append((x[i], y[j], z[k]))
        if isdistallocminima(data, x, y, z, i, j, k, min_distal, max_distal):
            distallocminima.append((x[i], y[j], z[k]))

    # Return results
    return localminima, distallocminima


In [ ]:
# Generate 3D topography data
x = np.linspace(-10, 10, 20)
y = np.linspace(-10, 10, 20)
z = np.linspace(-10, 10, 20)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
data = np.sin(np.sqrt(X**2 + Y**2 + Z**2)) * np.exp(-0.1*(X**2 + Y**2 + Z**2))

# Set parameters
window_size = 10
step_size = 1
min_distal = 2
max_distal = 4
n = 100

# Call function
islocalminima_3d(data, x, y, z, window_size, step_size, min_distal, max_distal, n)


ValueError: ignored